# Making a blog

## Inspired by Sam

He showed me the Jekyll template, but I might want to see if I can manage it using JupyterLab as I do most of my work in that environment.

I started searching for a JupyterLab method and found this post https://www.fast.ai/posts/2020-01-20-nb2md.html.  The blog points out that interactive widgets won't show up in the blog if done through Jupyter, which is unfortunate because I might like to make them.

I had seen nbdev mentioned before, which is why I looked into it instead of the other suggestion fast_template.  I first created the conda environment for nbdev-able JupyterLab.  I keep my JupyterLab installations in their own environment separate from the kernel environments as I find that to be the most organized way of working with them.  My current JupyterLab environments are just a standard JupyterLab, an SOS Notebook (story for another time, but I recommend it if you work in multiple languages frequently), and now this.

```bash
conda create -n Jupyter-nbdev
conda activate Jupyter-nbdev
```

Before installing packages I like to find out what are available.

```bash
conda search conda-forge::nbdev
# Name                       Version           Build  Channel             
nbdev                         2.2.10    pyhd8ed1ab_0  conda-forge  
```

Surprisingly there is only one version; typically there are more.  The official nbdev directions recommend installing from the fastai channel (`conda search fastai::nbdev`), which has multiple versions to choose from going up to 2.3.9.  I'm reluctant to use the fastai channel because I've had issues in the past with mixing channels - but, since I'm so early in the process I'll go for it.  Worst case scenario I'll kill this environment and start from scratch.

I'll install it along with Python 3.9 - I chose 3.9 because in the past not every library I've wanted has been compatible with 3.10, but I haven't encountered any conflicts with 3.9 yet.  I don't really need its features so I could be more conservative if needed.

```bash
conda install conda-forge::python=3.9 fastai::nbdev=2.3.9
```

I've heard that installing packages together in one line is better; I've never really bothered before but I'll give it a try here.  I don't think I'll notice a difference with the limited installs we'll be doing today.

I then remember I still need to install JupyterLab - so much for a one-liner.  I `conda search` it to determine the most recent version, and then install.  I like being explicit with my versioning, so that `conda env export --from-history` is more consistent across time.

```bash
conda install conda-forge::jupyterlab=3.5.1
```

Since I keep my JupyterLab and kernel environments separate, I create a new environment, exactly the same but without Jupyterlab.  According to the docs, nbdev needs to be installed in both.

```bash
# Inside (base) environment
conda create -n Blog-nbdev
conda activate Blog-nbdev
conda install conda-forge::python=3.9 fastai::nbdev=2.3.9
```

I want to make Blog-nbdev findable as a kernel by Jupyter-nbdev.  I've done this before, and remember that it goes something like this (backed up by StackOverflow https://stackoverflow.com/questions/39604271/conda-environments-not-showing-up-in-jupyter-notebook):

```bash
# (Jupyter-nbdev)
conda search conda-forge::nb_conda_kernels
conda install conda-forge::nb_conda_kernels=2.3.1

# (Blog-nbdev)
conda search conda-forge::ipykernel
conda install conda-forge::ipykernel=6.19.2
```


After that, I launch JupyterLab and get to work on a test example.

```bash
# (Jupyter-nbdev)
jupyter lab
```

I make sure to set the kernel to `Python [conda env:Blog-nbdev]`, and then I paste these notes in it.


I continue on to read about how to use nbdev with GitHub: https://nbdev.fast.ai/tutorials/tutorial.html#first-steps.  While doing this, I found out how to customize terminal setup (will change, just showing it off b/c I never knew you could have tabs and rename them.)  It's convenient because I can have different tabs for each of the different threads of this project.

<br>

![Customized Terminal](images/CustomizedTerminal.png)

I followed their instructions for setting up a GitHub repo,
and then set up nbdev as follows:

```bash
# In the directory of my cloned github repo.
conda activate Blog-nbdev
nbdev_new
```

I got this output message:
```bash
/Users/baileyandrew/opt/anaconda3/envs/Blog-nbdev/lib/python3.9/site-packages/ghapi/core.py:101: UserWarning: Neither GITHUB_TOKEN nor GITHUB_JWT_TOKEN found: running as unauthenticated
  else: warn('Neither GITHUB_TOKEN nor GITHUB_JWT_TOKEN found: running as unauthenticated')
repo = Blog # Automatically inferred from git
branch = main # Automatically inferred from git
user = BaileyAndrew # Automatically inferred from git
author = Bailey Andrew # Automatically inferred from git
author_email = e.bailey.andrew@gmail.com # Automatically inferred from git
description = A blog. # Automatically inferred from git
settings.ini created.
/bin/sh: quarto: command not found
```

It looks a little warning-y but passable.  Finally, I pushed the changes:

```bash
git add .
git commit -m 'Created Blog'
git push
```

When pushing, I got the failing error " ! [remote rejected] main -> main (refusing to allow a Personal Access Token to create or update workflow `.github/workflows/deploy.yaml` without `workflow` scope)".  So I guess I have to deal with those again.  They probably make things more secure, but they're a pain.

I always forget where to find these tokens: On GitHub, click on your profile and go to 'settings', then 'developer settings', 'personal access tokens', and finally 'tokens (classic)'.  The problem is that I have only ever created tokens with just 'repo' permissions, but it seems nbdev requires 'workflow' permissions.  I checked both boxes.

Then, I realized that you can actually update tokens, so after redundantly creating a new one I just went back and updated my original one to have that scope.

I then followed their directions to get to the Workflows part of Github, and clicked on the 'run workflow' dropdown here: ![workflow](./images/WhatToRun.png)

However this turned out to be the wrong thing to do 😅 .  I needed to go into settings first to set the branch to be `gh-pages`: ![fix](./images/SetupPages.png).

It isn't quite working yet because this file doesn't show up, but rather the default: ![darn](./images/FalseSuccess.png)

To enable updating the repo, I ran:

```bash
# (Blog-nbdev), inside local repo
nbdev_install_hooks
```

And I moved this notebook into the auto-generated 'nbs' folder in the repo.

It turned out that the main issue was not having a header - you need to have a header, which will be the title of the blog post.  Furthermore you need to ensure it's in its own Markdown block, otherwise the remaining text will not appear. The last issue was getting the images to display, which was overcome by creating an 'images' folder in the auto-generated 'nbs' folder and storing them there.

After that, it worked like a charm!  Running `nbdev_preview` was quite helpful in debugging the issues, as it previews your website and updates automatically!  I'll clean this blog up in the morning so that it's actually readable ;)